## Projet InPoDa :
Dans le câdre du TD11-12 nous devions travailler sur un programme permettant de fournir de la statistique sur des tweet récupérés sous format *json*.

In [6]:
#############################
### Import des librairies ###
import json, re
from textblob import TextBlob
import tkinter as tk #pas encore utilisé
import matplotlib #pas encore utilisé
#############################

In [2]:
###############################################################
###########################Fonctions###########################
def Tweet(data:dict):
    '''Prend un tweet sous format dictionnaire.\n 
    Procède à un tri des informations pour ne garder que l'essentiel.\n
    Renvoi un nouveau dictionnaire avec des données prêtes à l'emploi. '''
    self = {}
    self['text'] = data['text']
    self['cleaned_text'] = data['text']
    self['author'] = data['author_id']
    self['hashtags'] = []
    self['urls'] = []
    self['mentions'] = []
    if 'entities' in data.keys():
        if 'hashtags' in data['entities']:
            for i in data['entities']['hashtags']:
                if i['tag'] not in self['hashtags']:
                    self['hashtags'].append(i['tag'])
        if 'urls' in data['entities']:
            for i in data['entities']['urls']:
                if i['url'] not in self['urls']: 
                    self['urls'].append(i['url'])
                    self['cleaned_text'] = self['cleaned_text'].replace(i['url'],'')
        if 'mentions' in data['entities']:
            for i in data['entities']['mentions']:
                if i['id'] not in self['mentions']:
                    self['mentions'].append(i['id'])
    self['cleaned_text'] = re.sub(r'#\w+|@\w+|[^a-zA-Zàâçéèêëîïôûùüæœ\d\s]','',self['cleaned_text'])
    self['textblob'] = TextBlob(self['cleaned_text'])
    self['sentiment'] = self['textblob'].sentiment
    self['topics'] = list(self['textblob'].noun_phrases)
    return self
###############################################################

In [3]:
class InPoDa():
    def __init__(self, json_path):
        '''# Utilité :\n
        Permet d'initialiser une instance InPoDa selon un json
        contenant des tweets, vous pouvez ensuite appeler ses différentes méthodes
        pour accèder à différent stats.'''
        with open(json_path) as f:
            self.json = json.load(f)
        self.TweetData = []
        for raw in self.json:
            self.TweetData.append(Tweet(raw))
        self.pub_per_users = {}
        self.pub_per_tag = {}
        self.pub_per_topic = {}
        for i in self.TweetData:
            if i['author'] in self.pub_per_users.keys():
                self.pub_per_users[i['author']] += 1
            else:
                self.pub_per_users[i['author']] = 1
            for tag in i['hashtags']:
                if tag in self.pub_per_tag.keys():
                    self.pub_per_tag[tag] += 1
                else:
                    self.pub_per_tag[tag] = 1
            for topic in i['topics']:
                if topic in self.pub_per_topic.keys():
                    self.pub_per_topic[topic] += 1
                else:
                    self.pub_per_topic[topic] = 1

    
    def top_k_hashtags(self,k):
        '''Retourne le top k des Hashtags les plus utilisés dans l'instance'''
        return sorted(self.pub_per_tag.items(), key=lambda x:x[1],reverse=True)[:k]
    
    def top_k_user(self,k):
        '''Retourne le top k des utilisateurs les plus actifs dans l'instance'''
        return sorted(self.pub_per_users.items(), key=lambda x:x[1],reverse=True)[:k]
    
    def top_k_mentionned_user(self,k):
        '''Retourne le top k des utilisateurs les plus mentionnés dans l'instance'''
        mentions = {}
        for tweet in self.TweetData:
            for user in tweet['mentions']:
                if user in mentions.keys():
                    mentions[user] += 1
                else:
                    mentions[user] = 1
        return sorted(mentions.items(), key=lambda x:x[1],reverse=True)[:k]
    
    def top_k_topic(self,k):
        '''Retourne le top k des topics les plus mentionnés dans l'instance'''
        return sorted(self.pub_per_topic.items(), key=lambda x:x[1],reverse=True)[:k]
    
    def get_user_tweets(self,user_id:str):
        '''Retourne tout les tweets d'un utilisateur existant dans l'instance'''
        output = []
        for tweet in self.TweetData:
            if tweet['author'] == user_id:
                output.append(tweet)
        return output
    
    def get_tweets_user_mentionned(self,user_id:str):
        '''Retourne tout les tweets où utilisateur existant dans l'instance est mentionné'''
        output = []
        for tweet in self.TweetData:
            for m_id in tweet['mentions']:
                if m_id == user_id:
                    output.append(tweet)
        return output
    
    def get_users_by_hashtag(self,tag:str):
        '''Retourne tout les utilisateurs de ce Hashtag'''
        output = {}
        for tweet in self.TweetData:
            for hashtag in tweet['hashtags']: ##########
                if tag == hashtag:
                    if tweet['author'] not in output.keys():
                        output[tweet['author']] = 1
                    else:
                        output[tweet['author']] += 1
        return output
    
    def get_author_mentions(self,user_id:str):
        '''Retourne toutes les mentions que cet utilisateur a fait'''
        output = {}
        for tweet in self.TweetData:
            if tweet['author'] == user_id:
                for m in tweet['mentions']:
                    if m not in output.keys():
                        output[m] = 1
                    else:
                        output[m] += 1
        return output


## Démonstration à l'usage:

In [4]:
instance1 = InPoDa('versailles_tweets_100.json')

In [5]:
print(instance1.top_k_user(5))

[('1339914264522461187', 4), ('992904738516717570', 4), ('717025418', 2), ('3169236915', 2), ('372993152', 2)]


In [6]:
print(instance1.top_k_hashtags(5))

[('CIV', 2), ('twitter225', 1), ('SupportriceMazo', 1), ('domie', 1), ('jifa', 1)]


In [7]:
print(instance1.top_k_mentionned_user(5))

[('3200704501', 3), ('19811019', 2), ('4827016745', 1), ('254068589', 1), ('781489936184651776', 1)]


In [8]:
print(instance1.top_k_topic(5))

[('versailles', 4), ('goumin', 1), ('des lphants joueurs', 1), ('mme fatigue mme', 1), ('mes tontons vous avez fait votre part', 1)]


In [9]:
print(instance1.pub_per_users)

{'1339914264522461187': 4, '717025418': 2, '992904738516717570': 4, '736523371': 1, '1471684208': 1, '3169236915': 2, '16267684': 1, '60117154': 1, '372993152': 2, '105241852': 1, '2357913366': 1}


In [10]:
print(instance1.pub_per_tag)

{'twitter225': 1, 'SupportriceMazo': 1, 'domie': 1, 'CIV': 2, 'jifa': 1, 'versailles': 1, 'nocturne': 1, 'appollon': 1}


In [11]:
print(instance1.pub_per_topic)

{'goumin': 1, 'des lphants joueurs': 1, 'mme fatigue mme': 1, 'mes tontons vous avez fait votre part': 1, 'jo': 1, 'final au moins': 1, 'bravo': 1, 'sommeil l sera compliqu est limin des': 1, 'jo ahi': 1, 'peut faire': 1, 'juillet journe internationale': 1, 'femme africaine âï': 1, 'vous': 1, 'avez tt fait raison': 1, 'silence incomprhensible du gouvernoument': 1, 'des merdias sur ce trs': 1, 'dramatique sujet prouve': 1, 'irrfutable leur implication ce plan diabolique maquill': 1, 'grande muette': 1, 'rester part quelques irrductibles': 1, 'les': 1, 'bains dapollon': 1, 'chteau': 1, 'versailles': 4, 'rdv aujourdhui aura tenu ses promessesâïââ': 1, 'pour': 1, 'les problmes': 1, 'je dors comme un bb': 1, 'rveil': 1, 'pleine forme assur': 1, 'et': 1, 'sans acoutumances pas comme avec ces bquilles chimiquesâ': 1, 'bonne': 1, 'julie': 1, 'il est temps': 1, 'laisser mijoter': 1, 'un': 1, 'mouton cest bien': 1, 'intelligent que toi': 1, 'im': 1, 'gardens': 1, 'jungle cruise ugc cyrano': 1, '

In [12]:
print(instance1.get_user_tweets('992904738516717570'))

[{'text': '@isabelle170516 @leonna_julie @Steiner2502 Vous avez tt Ã\xa0 fait raison! le silence incomprÃ©hensible du gouver-noument et des merdias sur ce trÃ¨s important et dramatique sujet prouve de maniÃ¨re irrÃ©futable\n leur implication Ã\xa0 ce plan  diabolique maquillÃ©!', 'cleaned_text': '   Vous avez tt \xa0 fait raison le silence incomprhensible du gouvernoument et des merdias sur ce trs important et dramatique sujet prouve de manire irrfutable\n leur implication \xa0 ce plan  diabolique maquill', 'author': '992904738516717570', 'hashtags': [], 'urls': [], 'mentions': ['781489936184651776', '3200704501', '1246352652700659713'], 'textblob': TextBlob("   Vous avez tt   fait raison le silence incomprhensible du gouvernoument et des merdias sur ce trs important et dramatique sujet prouve de manire irrfutable
 leur implication   ce plan  diabolique maquill"), 'sentiment': Sentiment(polarity=0.4, subjectivity=1.0), 'topics': ['vous', 'avez tt fait raison', 'silence incomprhensible 

In [13]:
print(instance1.get_tweets_user_mentionned('3200704501'))

[{'text': '@isabelle170516 @leonna_julie @Steiner2502 Vous avez tt Ã\xa0 fait raison! le silence incomprÃ©hensible du gouver-noument et des merdias sur ce trÃ¨s important et dramatique sujet prouve de maniÃ¨re irrÃ©futable\n leur implication Ã\xa0 ce plan  diabolique maquillÃ©!', 'cleaned_text': '   Vous avez tt \xa0 fait raison le silence incomprhensible du gouvernoument et des merdias sur ce trs important et dramatique sujet prouve de manire irrfutable\n leur implication \xa0 ce plan  diabolique maquill', 'author': '992904738516717570', 'hashtags': [], 'urls': [], 'mentions': ['781489936184651776', '3200704501', '1246352652700659713'], 'textblob': TextBlob("   Vous avez tt   fait raison le silence incomprhensible du gouvernoument et des merdias sur ce trs important et dramatique sujet prouve de manire irrfutable
 leur implication   ce plan  diabolique maquill"), 'sentiment': Sentiment(polarity=0.4, subjectivity=1.0), 'topics': ['vous', 'avez tt fait raison', 'silence incomprhensible 

In [14]:
print(instance1.get_users_by_hashtag('CIV'))

{'1339914264522461187': 2}


In [15]:
print(instance1.get_author_mentions('992904738516717570'))

{'781489936184651776': 1, '3200704501': 3, '1246352652700659713': 1, '1355767640036438016': 1, '1071056487278104577': 1, '4216955975': 1}


## Interface Utilisateur du Programme :

In [8]:
root = tk.Tk()
root.title("InPoDa")
root.geometry('800x800')
root.mainloop()